## 1. Visualización imágenes y registro EEG

In [ ]:
import random
import os
from os import listdir
from os.path import isfile, join
import subprocess, sys
import pandas as pd
import numpy as np
import time

In [ ]:
# DESDE LA TERMINAL

# $ muselsl list                       --> Detectar Muse
# $ muselsl stream --name MUSE-96E2    --> Comenzar stream
# $ muselsl view --version 2           --> Ver registro, comprobar buena señal, sin artefactos. doble pantalla
# $ muselsl record --duration 900     --> Comenzar a grabar. Duracion en segundos

In [ ]:
# lista con todos los nombres de las imágenes: path to the folder with the pictures

list_pictures = [f for f in listdir('/todas_imagenes') 
                 if isfile(join('/todas_imagenes', f))]

In [ ]:
# Crear la lista con las imágenes. (730 imágenes* 10s por imagen + 2s imágen negra) --> lista completa
# Con la lista completa hacer un shuffle y subdividir en 10 listas.--> 73 imágenes por lista

def chunkIt(lista, n):                                     #coge la lista y la divide entre n
    avg = len(lista) / float(n)
    out = []
    last = 0.0
    while last < len(lista):
        out.append(lista[int(last):int(last + avg)])
        last += avg
    return out

random.shuffle(list_pictures)
listas_n = chunkIt(list_pictures, 10)

In [ ]:
# ESTE PASO SOLO HACER UNA VEZ! SE GUARDA LA LISTA CON LAS IMÁGENES ALEATORIAS Y DESPUES SE CARGA LA LISTA

for lista in listas_n:                          #guarda un txt con todas las listas creadas
    mi_path = "./lista_de_listas.txt"
    with open(mi_path, 'a+') as f:
        f.write(str(lista)+ '\n')
        
lista_1 = listas_n[0]
lista_2 = listas_n[1]
lista_3 = listas_n[2]
lista_4 = listas_n[3]
lista_5 = listas_n[4]
lista_6 = listas_n[5]
lista_7 = listas_n[6]
lista_8 = listas_n[7]
lista_9 = listas_n[8]
lista_10 = listas_n[9]

In [ ]:
# De cada una de las listas formadas en el paso anterior, va abriendo sus elementos, cada foto está 10 segundos, 
# despues 2s de pantalla negra, abre otra imagen 10 segundos y así sucesivamente. Crea un dict con imagen:timestamp

def visualize_pics (lista):
    pic_time= {}
    for p in lista:
        opener ="open" if sys.platform == "darwin" else "xdg-open"
        subprocess.call([opener, p])
        timestamp1 = int(round(time.time() * 1000))
        time.sleep(10)                                                   # tiempo que está la foto
        subprocess.call([opener, 'black-screen.png'])
        time.sleep(2)                                                    #tiempo pantalla negra  
        timestamp2 = int(round(time.time() * 1000))                              
        pic_time.update({p:(timestamp1,timestamp2)})
    pic_time_df = pd.DataFrame(list(pic_time.items()), columns=['Picture', 'timestamps'])
    pic_time_df = pic_time_df.join(pd.DataFrame(pic_time_df['timestamps'].values.tolist(), columns=['start_time', 'end_time']))
    pic_time_df = pic_time_df.drop(['timestamps'], axis = 1)
    return pic_time_df                             

In [ ]:
# Dataset con los datos de imagen y timestamp start y end
# n es un numero del 1 al 10

pic_time_df = visualize_pics(lista_n) 
pic_time_df.to_csv('pic_time_n',index=False) 

In [ ]:
# Aqui imprimimos el dataset con los datos de la grabación EEG
# el csv que emite Muse hay que abrirlo primero con excel y guardarlo como csv para que salgan bien los timestamps
# GUARDAR CON EXCEL CON 'registro + NUMERO.csv'(numero: 1-10)

df_registro = pd.read_csv('./registro_n.csv')           #record grabado durante las imagenes
df_registro.drop(['Right AUX'], axis=1, inplace=True)   # borrar canal que no se utiliza
df_registro['image']='pic'                              # Crear la columna image con un default value

- Merge entre dataset pic_time_df (imagen-start-end) y el df_registro EEG. Para cada row del EEG añadir el nombre de imagen que corresponda, que será aquella cuyo start y en time engloben al timestamp de la row de EEG.
    - DF resultante: imagen-timestamp-canal1-canal2-canal3-canal4. 
    - Cada imagen aparecerá en unas 3080 filas
    - Tendré 10 dataset así, uno por cada grupo de imágenes. 

- Finalmente concat de los 10 dataset--> Dataset 2540* 730; 1854200 filas y 6 columnas

In [ ]:
# Asignar a cada row del EEG la imagen que corresponde

for row in pic_time_df.iterrows():
    df_registro.loc[(df_registro['timestamps']> row[1].start_time) & 
                    (df_registro['timestamps'] < row[1].end_time), 'image'] = row[1].Picture

df_registro_pic = df_registro[df_registro.image != 'pic'] # Borrar rows sin imagen asignada

In [ ]:
# De estos tendré un total de 10, que tendré que concatenar

df_registro_pic.to_csv('registro_pic_n.csv',index=False)   # NUMERO 1-10